In [1]:
%use fuel(2.3.1)

In [2]:
val envs = java.io.File("../../.env")
    .readLines()
    .map {
        it.split("=")[0] to it.split("=")[1].trim('"')
    }.toMap()

In [3]:
val session = envs.get("AOC_SESSION")
val year = 2021
val day = 19

In [4]:
fun getInput(year: Int, day: Int, session: String): String {
    val (_, _, result) = "https://adventofcode.com/$year/day/$day/input"
    .httpGet()
    .header("cookie" to "session=$session")
    .responseString()
        
    return result.get().trim()
}

In [5]:
fun submitAnswer(year: Int, day: Int, session: String, level: Int, answer: String): String {
    val (_, _, result) = Fuel
    .post(
        "https://adventofcode.com/$year/day/$day/answer", 
        parameters = listOf("level" to level, "answer" to answer))
    .header("cookie" to "session=$session")
    .responseString()
        
    return result.get()
}

In [6]:
val sample = """--- scanner 0 ---
404,-588,-901
528,-643,409
-838,591,734
390,-675,-793
-537,-823,-458
-485,-357,347
-345,-311,381
-661,-816,-575
-876,649,763
-618,-824,-621
553,345,-567
474,580,667
-447,-329,318
-584,868,-557
544,-627,-890
564,392,-477
455,729,728
-892,524,684
-689,845,-530
423,-701,434
7,-33,-71
630,319,-379
443,580,662
-789,900,-551
459,-707,401

--- scanner 1 ---
686,422,578
605,423,415
515,917,-361
-336,658,858
95,138,22
-476,619,847
-340,-569,-846
567,-361,727
-460,603,-452
669,-402,600
729,430,532
-500,-761,534
-322,571,750
-466,-666,-811
-429,-592,574
-355,545,-477
703,-491,-529
-328,-685,520
413,935,-424
-391,539,-444
586,-435,557
-364,-763,-893
807,-499,-711
755,-354,-619
553,889,-390

--- scanner 2 ---
649,640,665
682,-795,504
-784,533,-524
-644,584,-595
-588,-843,648
-30,6,44
-674,560,763
500,723,-460
609,671,-379
-555,-800,653
-675,-892,-343
697,-426,-610
578,704,681
493,664,-388
-671,-858,530
-667,343,800
571,-461,-707
-138,-166,112
-889,563,-600
646,-828,498
640,759,510
-630,509,768
-681,-892,-333
673,-379,-804
-742,-814,-386
577,-820,562

--- scanner 3 ---
-589,542,597
605,-692,669
-500,565,-823
-660,373,557
-458,-679,-417
-488,449,543
-626,468,-788
338,-750,-386
528,-832,-391
562,-778,733
-938,-730,414
543,643,-506
-524,371,-870
407,773,750
-104,29,83
378,-903,-323
-778,-728,485
426,699,580
-438,-605,-362
-469,-447,-387
509,732,623
647,635,-688
-868,-804,481
614,-800,639
595,780,-596

--- scanner 4 ---
727,592,562
-293,-554,779
441,611,-461
-714,465,-776
-743,427,-804
-660,-479,-426
832,-632,460
927,-485,-438
408,393,-506
466,436,-512
110,16,151
-258,-428,682
-393,719,612
-211,-452,876
808,-476,-593
-575,615,604
-485,667,467
-680,325,-822
-627,-443,-432
872,-547,-609
833,512,582
807,604,487
839,-516,451
891,-625,532
-652,-548,-490
30,-46,-14"""

In [7]:
val input = getInput(year, day, session)

In [8]:
fun solve(input: String): String {
    val scanners = input
        .split(Regex("""--- scanner \d+ ---\n"""))
        .drop(1)
        .map {
            it
                .split("\n")
                .filter { it.length > 0 }
                .map { it.split(",").map(String::toInt) }
        }
        .toMutableList()
    
    val n = scanners.size
    val positions = Array(n) { IntArray(3) }
    val fixed = BooleanArray(n)
    fixed[0] = true
    
    fun overlap(i: Int, j: Int): Boolean {
        val si = scanners[i].toSet()
        
        for (p in listOf(listOf(0, 1, 2), listOf(0, 2, 1), listOf(1, 0, 2), listOf(1, 2, 0), listOf(2, 0, 1), listOf(2, 1, 0)))
            for (c in listOf(listOf(-1, -1, -1), listOf(-1, -1, 1), listOf(-1, 1, -1), listOf(-1, 1, 1), listOf(1, -1, -1), listOf(1, -1, 1), listOf(1, 1, -1), listOf(1, 1, 1))) {
                val t = scanners[j].map {
                    beacon -> (0..2).map { c[it] * beacon[p[it]] }
                }
                
                for (u in scanners[i])
                    for (v in t) {
                        val tt = t.map {
                            beacon -> (0..2).map { beacon[it] - (v[it] - u[it]) }
                        }
                        val sij = tt.toSet().intersect(si)
                        
                        if (sij.size >= 12) {
                            fixed[j] = true
                            scanners[j] = tt
                            positions[j] = (0..2).map { v[it] - u[it] }.toIntArray()
                            return true
                        }
                    }
            }
        
        return false
    }
    
    for (t in 0 until n - 1) {
        var found = false
        
        for (i in 0 until n) {
            if (!fixed[i]) continue
            for (j in 0 until n) {
                if (fixed[j]) continue
                if (overlap(i, j)) {
                    found = true
                    break
                }
            }
            if (found) break
        }
    }
    
    var beacons = scanners[0].toSet()
    for (i in (1 until n))
        beacons = beacons.union(scanners[i].toSet())
    val numberOfBeacons = beacons.size
    
    var maxDistance = 0
    for (u in positions)
        for (v in positions)
            maxDistance = maxDistance.coerceAtLeast((0..2).map { abs(u[it] - v[it]) }.sum())
    
    return Pair(numberOfBeacons, maxDistance).toString()
}

In [9]:
solve(sample)

(79, 3621)

In [10]:
solve(input)

(440, 13382)